In [1]:
import string
import matplotlib.pyplot as plt

In [2]:
# We could have a machine language with only NAND.
# But this will cause an exponential blowup in the term length.
# To not get an exponential blowup, we need at least NOT, AND, IMPLIES, and IFF.
# (Or something equivalent, e.g. we could have XOR instead of AND.)
# On the other hand, compiling ∃ to ¬∀¬ does not cause any blowup.

TOKENS = [
    ('LPAREN', '('),
    ('RPAREN', ')'),
    ('NOT', r'\neg'),
    ('AND', r'\wedge'),
    ('IMPLIES', r'\implies'),
    ('IFF', r'\iff'),
    ('FORALL', r'\forall'),
    ('CONTAINS', r'\in'),
    ####################
    ('OR', r'\vee'),
    ('UNION', r'\cup'),
    ('INTERSECTION', r'\cap'),
    ('EXISTS', r'\exists'),
    ('SUBSET', r'\subset'),
    ('EQUALS', '='),
    ###################
    ('FORALL', '∀'),
    ('EXISTS', '∃'),
    ('NOT', '¬'),
    ('CONTAINS', '∈'),
    ('AND', '∧'),
    ('OR', '∨'),
    ('IMPLIES', '⇒'),
    ('IFF', '⇔'),
    ('SUBSET', '⊂'),
#     ('NOT', r'\not'),
]

def tokenize(formula):
    formula = '(%s)' % formula
    def helper():
        i = 0
        while i < len(formula):
            if formula[i] == ' ':
                i += 1
                continue
            for t, v in TOKENS:
                if formula[i:i+len(v)] == v:
                    yield (t, None)
                    i += len(v)
                    break
            else:
                assert formula[i] in string.ascii_lowercase, (i, formula[i])
                yield ('VARIABLE', formula[i])
                i += 1
    return list(helper())

In [3]:
BINOPS = ['OR', 'AND', 'IMPLIES', 'CONTAINS', 'SUBSET', 'EQUALS', 'IFF']
QUANTS = ['FORALL', 'EXISTS']
PRINT_SYMS = {
    'OR': '∨',
    'AND': '∧',
    'IMPLIES': '⇒',
    'IFF': '⇔',
    'FORALL': '∀',
    'EXISTS': '∃',
    'CONTAINS': '∈',
    'SUBSET': '⊂',
    'NOT': '¬',
    'EQUALS': '=',
}

class ASTNode:
    def __init__(self, node_type, node_value=None):
        self.t = node_type
        self.v = node_value

    def add_child(self, node):
        self.v.append(node)
    
    def __repr__(self):
        if isinstance(self.v, list):
            if self.t in BINOPS:
                return '(%s %s %s)' % (self.v[0], PRINT_SYMS[self.t], self.v[1])
            if self.t in QUANTS:
                return '%s%s%s' % (PRINT_SYMS[self.t], self.v[0], self.v[1])
            if self.t == 'NOT':
                return '%s%s' % (PRINT_SYMS[self.t], self.v[0])
            else:
                return '%s(%s)' % (self.t, ' '.join(str(n) for n in self.v))
        else:
            return str(self.v if self.t == 'VARIABLE' else self.t)

def parse(tokens):
    def parse_expression(index):
        if tokens[index][0] == 'LPAREN':
            index += 1
            node = ASTNode('GROUP', [])
            while tokens[index][0] != 'RPAREN':
                child_node, index = parse_expression(index)
                node.add_child(child_node)
            return node, index + 1 # skip RPAREN
        else:
            return ASTNode(*tokens[index]), index+1

    ast_root, _ = parse_expression(0)
    return ast_root

In [4]:
# infers the proper grouping for not and forall/exists
def group_terms(node):
    def helper(node):
        if not isinstance(node.v, list):
            return node
        children = node.v = [helper(n) for n in node.v]
        
        def group_at(i): # tries to form a group at index i
            if i >= len(children)-1:
                return
            
            if children[i].t == 'NOT':
                op_node = children.pop(i)
                group_at(i)
                a_node = children.pop(i)
                assert a_node.t == 'GROUP', a_node
                children.insert(i, ASTNode('GROUP', [op_node, a_node]))
                
            if children[i].t in ['FORALL', 'EXISTS']:
                op_node = children.pop(i)
                x_node = children.pop(i)
                assert x_node.t == 'VARIABLE'
                group_at(i)
                phi_node = children.pop(i)
                assert phi_node.t == 'GROUP'
                children.insert(i, ASTNode('GROUP', [op_node, x_node, phi_node]))
                
        i = 0
        while i < len(children):
            group_at(i)
            i += 1
        
        if len(children) == 1:
            return children[0] # eliminate unary groups
        else:
            node.v = children
            return node
        
    return helper(node)

In [5]:
def transform_noop(node):
    if not isinstance(node.v, list):
        return node
    children = node.v = [transform_noop(n) for n in node.v]
    return node
    
def extract_quantifiers(node):
    if not isinstance(node.v, list):
        return node
    children = node.v = [extract_quantifiers(n) for n in node.v]
    if node.t == 'GROUP':
        if children[0].t in ['FORALL', 'EXISTS'] and not children[0].v:
            assert children[1].t == 'VARIABLE', children
            assert len(children) == 3
            node.t = children[0].t
            node.v = [children[1], children[2]]
    return node

def extract_binops(node):
    def helper(node):
        if not isinstance(node.v, list):
            return node
        children = node.v = [helper(n) for n in node.v]
        if node.t == 'GROUP':
            if children[1].t in BINOPS and not children[1].v:
                assert len(children) == 3
                node.t = children[1].t
                node.v = [children[0], children[2]]
        return node
    return helper(node)

def extract_not(node):
    def helper(node):
        if not isinstance(node.v, list):
            return node
        children = node.v = [helper(n) for n in node.v]
        if node.t == 'GROUP':
            if children[0].t == 'NOT' and not children[0].v:
                assert len(children) == 2, children
                node.t = children[0].t
                node.v = [children[1]]
        return node
    return helper(node)

In [6]:
# check the formula for basic validity: e.g., (x ∈ (y ∈ x)) is invalid
def tag_logic(node):
    def helper(node):
        if isinstance(node.v, list):
            for n in node.v:
                helper(n)
        if node.t == 'VARIABLE':
            node.is_logic = False
        elif node.t in ['NOT', 'AND', 'OR', 'IMPLIES', 'IFF']:
            assert all(n.is_logic for n in node.v)
            node.is_logic = True
        elif node.t in ['EXISTS', 'FORALL']:
            assert node.v[0].t == 'VARIABLE' # this is already known anyway
            assert node.v[1].is_logic
            node.is_logic = True
#         elif node.t in ['UNION', 'INTERSECTION']:
#             assert all(not n.is_logic for n in node.v)
#             node.is_logic = False
        elif node.t in ['CONTAINS', 'SUBSET', 'EQUALS']:
            assert all(n.t == 'VARIABLE' for n in node.v)
            node.is_logic = True
        else:
            assert False
    helper(node)
    assert node.is_logic

In [7]:
def process(tree):
    tree = group_terms(tree)
    tree = extract_quantifiers(tree)
    tree = extract_binops(tree)
    tree = extract_not(tree)
    return tree

In [8]:
# # optional zero-level rules which cause an exponential blowup in formula length, and the duplication of
# # quantified terms.
# def compile_level0(tree):
#     def helper(node):
#         if not isinstance(node.v, list):
#             return node
#         node.v = [helper(n) for n in node.v]

#         if node.t == 'IMPLIES':
#             a, b = node.v
#             node = ASTNode('NOT', [ASTNode('AND', [a, ASTNode('NOT', [b])])])
            
#         if node.t == 'IFF':
#             a, b = node.v
#             not1 = ASTNode('NOT', [ASTNode('AND', [a, ASTNode('NOT', [b])])])
#             not2 = ASTNode('NOT', [ASTNode('AND', [b, ASTNode('NOT', [a])])])
#             node = ASTNode('AND', [not1, not2])
            
#         return node
#     return helper(tree)

# first-level compilation rules
def compile_level1(tree):
    def helper(node):
        if not isinstance(node.v, list):
            return node
        node.v = [helper(n) for n in node.v]
        
        if node.t == 'OR':
            a, b = node.v
            node = ASTNode('NOT', [ASTNode('AND', [ASTNode('NOT', [a]), ASTNode('NOT', [b])])])

        if node.t == 'EXISTS':
            x, phi = node.v
            node = ASTNode('NOT', [ASTNode('FORALL', [x, ASTNode('NOT', [phi])])])
            
        return node
    return helper(tree)

# second level compilation rules which depend on the first level
def compile_level2(tree):
    
    var_index = [0]
    def get_var():
        var_index[0] += 1
        return 'c%d' % var_index[0]
    
    def helper(node):
        if not isinstance(node.v, list):
            return node
        node.v = [helper(n) for n in node.v]

        if node.t == 'SUBSET':
            a, b = node.v
            v = get_var()
            imp = ASTNode('IMPLIES', [ASTNode('CONTAINS', [ASTNode('VARIABLE', v), a]),
                                      ASTNode('CONTAINS', [ASTNode('VARIABLE', v), b])])
            node = ASTNode('FORALL', [ASTNode('VARIABLE', v), imp])

        # compilation-time axiom of extensionality
        if node.t == 'EQUALS':
            a, b = node.v
            v = get_var()
            iff = ASTNode('IFF', [ASTNode('CONTAINS', [ASTNode('VARIABLE', v), a]),
                                  ASTNode('CONTAINS', [ASTNode('VARIABLE', v), b])])
            node = ASTNode('FORALL', [ASTNode('VARIABLE', v), iff])
            
        return node
    return helper(tree)

In [9]:
def tag_quantification(node):
    """
    check the formula for quantification validity: e.g., the following are invalid
    - ∀x∀x(x ∈ x) ; something already quantified is being quantified again
    - (x ∈ y) ; everything needs to be quantified
    - ∀x(x ∈ x) ∧ ∀x(x ∈ x) ; quantified variables should be unique.
    """
    def helper(node):
        if isinstance(node.v, list):
            for n in node.v:
                helper(n)
        if node.t == 'VARIABLE':
            node.free_vars = [node.v]
            node.quantified_vars = []
        elif node.t in ['NOT', 'AND', 'OR', 'IMPLIES', 'IFF', 'CONTAINS']: # 'CONTAINS', 'SUBSET', 'EQUALS'
            node.free_vars = [v for n in node.v for v in n.free_vars]
            node.quantified_vars = [v for n in node.v for v in n.quantified_vars]
            assert len(node.quantified_vars) == len(set(node.quantified_vars)) # Unique quantififed variables
        elif node.t in ['EXISTS', 'FORALL']:
            assert node.v[0].t == 'VARIABLE'
            q_var = node.v[0].v
            subtree_free = [v for n in node.v for v in n.free_vars]
            subtree_quantified = [v for n in node.v for v in n.quantified_vars]
#             assert q_var in subtree_free # commented out because it's OK to quantify things that aren't free.
            assert q_var not in subtree_quantified
            node.free_vars = [v for v in subtree_free if v != q_var]
            node.quantified_vars = [q_var] + subtree_quantified
        else:
            assert False, node
    helper(node)
    assert not node.free_vars

In [10]:
# Axiom of the empty set
# formula = r'\exists x (\forall y (\not (y \in x)))'
# formula = r'∃x∀y¬(y ∈ x)'

# Theorem: every set contains the empty set
# formula = r'\forall x ((\forall y (\neg (y \in x))) \implies (\forall a (x \subset a)))'
# formula = r'∀x(∀y¬(y ∈ x) ⇒ ∀a(x ⊂ a))'

# no two-cycles
# formula = r'∀x∀y \not ((x \in y) \wedge (y \in x))'

# Axiom of powerset
# formula = r'∀x∃y∀z((z ⊂ x) ⇒ (z ∈ y))'

# formula = '∀x∀y((x = y) \implies (x ⊂ y))'

# A provable tautology.
# formula = '∀x∀y((x = y) \iff ((x ⊂ y) \wedge (y ⊂ x)))'
formula = '∀x∀y((x = y) ⇔ ((x ⊂ y) ∧ (y ⊂ x)))'

# formula = r'(((x \in a) \vee (x \in b)) \wedge \neg (x \in a)) \implies (x \in b)'
# formula = r'((x \in a) \wedge \neg (x \in a))'

print(formula)
tokens = tokenize(formula)
tree = parse(tokens)
tree = process(tree)
tag_logic(tree)
print(tree)
# tree = compile_machinecode(tree)
tree = compile_level2(tree)
tree = compile_level1(tree)
# tree = compile_level0(tree)
tag_quantification(tree)
print(tree)
print(tree.quantified_vars)

∀x∀y((x = y) ⇔ ((x ⊂ y) ∧ (y ⊂ x)))
∀x∀y((x = y) ⇔ ((x ⊂ y) ∧ (y ⊂ x)))
∀x∀y(∀c1((c1 ∈ x) ⇔ (c1 ∈ y)) ⇔ (∀c2((c2 ∈ x) ⇒ (c2 ∈ y)) ∧ ∀c3((c3 ∈ y) ⇒ (c3 ∈ x))))
['x', 'y', 'c1', 'c2', 'c3']


In [22]:
"""
We would like a system of deduction rules which generate theorems.
Where all theorems are of the form "[subset of ZF] => [Theorem]".
We cannot include all of ZF in the left hand side because of the (infinite) specification schema.


I. Properties of ∧, ¬.

RULE 1.
φ
----------------------
¬¬φ
----------------------
φ

RULE 2.
φ
----------------------
φ ∧ φ
----------------------
φ

RULE 3.
φ ∧ Ψ
----------------------
Ψ ∧ φ


II. Properties of ∀.
Summary: "∀x∀yφ(x, y) ⇔ ∀y∀xφ(x, y)" and "∀x∀yφ(x, y) ⇒ ∀xφ(x, x)" and "∀xφ(x) ⇔ ∀yφ(y)"

RULE 1.
∀x∀yφ(x, y)
----------------------
∀y∀xφ(x, y)

RULE 2.
∀x∀yφ(x, y)
----------------------
∀xφ(x, x)

RULE 3.
∀xφ(x)
----------------------
∀yφ(y)



III. Interactions between ∀, ∧, ¬.

RULE 1.
∀x¬φ(x)
----------------------
¬∀xφ(x)

RULE 2.
∀x∀y(φ(x) ∧ Ψ(x, y))
----------------------
∀x(φ(x) ∧ ∀yΨ(x, y))
----------------------
∀x∀y(φ(x) ∧ Ψ(x, y))

RULE 3.
∀x∀y¬(φ(x) ∧ Ψ(x, y))
----------------------
∀x¬(φ(x) ∧ ¬∀y¬Ψ(x, y))
----------------------
∀x∀y¬(φ(x) ∧ Ψ(x, y))


"""
pass

In [13]:
"""
Some meta-theorems (for arbitrary φ, Ψ).


THEOREM.
∀x∀y(φ(x) ∨ Ψ(x, y))
----------------------------------------
∀x∀y¬(¬φ(x) ∧ ¬Ψ(x, y))
----------------------------------------
∀x¬(¬φ(x) ∧ ¬∀y¬¬Ψ(x, y))
----------------------------------------
∀x¬(¬φ(x) ∧ ¬∀yΨ(x, y))
----------------------------------------
∀x(φ(x) ∨ ∀yΨ(x, y))
----------------------------------------


THEOREM.
φ ∨ Ψ
----------------------------------------
¬(¬φ ∧ ¬Ψ)
----------------------------------------
¬(¬Ψ ∧ ¬φ)
----------------------------------------
Ψ ∨ φ


THEOREM.
∀x∀y(φ(x) ∧ Ψ(y))
----------------------------------------
∀x(φ(x) ∧ ∀yΨ(y))
----------------------------------------
∀x(∀yΨ(y) ∧ φ(x))
----------------------------------------
∀yΨ(y) ∧ ∀xφ(x)
----------------------------------------


THEOREM.
∀x∀y(φ(x) ⇒ Ψ(x, y))
----------------------------------------
∀x∀y(¬φ(x) ∨ Ψ(x, y))
----------------------------------------
∀x(¬φ(x) ∨ ∀yΨ(x, y))
----------------------------------------




THEOREM 2. [meta-theorem for arbitrary φ, Ψ]
∀x∀y(φ(x) ⇒ Ψ(y))
----------------------
∀x∀y(¬φ(x) ∨ Ψ(y))
----------------------
[apply theorem 1]
----------------------
∃x¬φ(x) ∨ ∃yΨ(y)
----------------------
¬∀x¬¬φ(x) ∨ ∃yΨ(y)
----------------------
¬∀xφ(x) ∨ ∃yΨ(y)
----------------------





----------------------
∀x∀y¬(¬φ(x) ∧ ¬Ψ(y))
----------------------
∀x¬∀y(¬φ(x) ∧ ¬Ψ(y))
----------------------
∀x¬(¬φ(x) ∧ ∀y¬Ψ(y))
----------------------
¬∀x(¬φ(x) ∧ ∀y¬Ψ(y))
----------------------
¬(∀x¬φ(x) ∧ ∀y¬Ψ(y))
----------------------
¬∀x¬φ(x) ∨ ¬∀y¬Ψ(y)
----------------------
∃xφ(x) ∨ ∃yΨ(y)





----------------------
¬∀x∀y(¬φ(x) ∧ ¬Ψ(y))
----------------------
¬(∀x¬φ(x) ∧ ∀y¬Ψ(y))
----------------------
¬∀x¬φ(x) ∧ ¬∀y¬Ψ(y)
----------------------





∀x∀y(φ(x) ∨ Ψ(y))
----------------------
∀xφ(x) ∨ ∀yΨ(y)



∀x∀y¬(¬φ(x) ∧ ¬Ψ(y))
----------------------
¬(¬∀xφ(x) ∧ ¬∀yΨ(y))



¬(¬∀xφ(x) ∧ ¬∀yΨ(y))
----------------------
¬(¬∀xφ(x) ∧ ¬∀yΨ(y))




IV. Interaction between ∀ and ⇒.

These can be derived from the previous interactions via:
∀x∀y(φ(x) ⇒ Ψ(y))
----------------------
∀x∀y¬(φ(x) ∧ ¬Ψ(y))
----------------------
∀x∀y¬(φ(x) ∧ ¬Ψ(y))




RULE 5.
∀x∀y(φ(x) ⇒ Ψ(y))
----------------------
∀xφ(x) ∧ ∀yΨ(y)

RULE 6.
∀xφ(x) ∧ ∀yΨ(y)
----------------------
∀x∀y(φ(x) ∧ Ψ(y))


"""
pass